# AR Assistant: Documentación del Pipeline

En este Notebook describiremos el pipeline de nuestro Asistente de Realidad Aumentada (AR), el cual gestiona una interacción completa de inicio a fin, integrando tecnologías de procesamiento de voz (Speech-to-Text, Text-to-Speech) y visión por computadora (Detección de Objetos, Procesamiento de Vídeo y Q&A basado en Visión).

## Uso principal

El *pipeline* completo se ejecuta simplemente creando una instancia de la clase ARAssistant y llamándola.

```python
if __name__ == "__main__":
    assistant = ARAssistant()
    assistant()
```

## Estructura del Pipeline ARAssistant.__call__

### Inicio y Preparación
Inicialmente esperamos la orden del usuario transcribiendo el audio de entrada (Whisper) para detectar comandos como ["empezar", "juego"]. Al detectarlo, explicamos el juego usando Text-to-Speech (TTS). Luego, procesamos el video del entorno con YOLO para detectar objetos. Seleccionamos un frame aleatorio y de allí elegimos un único objeto secreto con una ponderación basada en el accuracy de cada detección.

### Bucle de Juego (Q&A)
Una vez procesado el vídeo iniciamos el bucle Q&A simulando una secuencia de audio lineal. Por cada pregunta, la transcribimos y verificamos si el texto contiene el nombre del objeto secreto para una victoria inmediata. Si no acierta, almacenamos la pregunta en una lista.

**Generación de Respuesta:** Cada dos preguntas, usamos el modelo CLIP para comparar la imagen del objeto con las preguntas almacenadas. La pregunta con la mayor accuracy (similitud) se selecciona como la respuesta más relevante, la cual reproducimos al usuario.

**Generación de Pista:** Cada seis preguntas (cada tres pares), generamos una pista. Elegimos un atributo predefinido ("color", "forma", ...) y usamos CLIP para relacionar la imagen con las posibles descripciones de ese atributo. La descripción con mayor probabilidad se da como pista al usuario, incluyendo el porcentaje de confianza.


```python
def __call__(self):
        # pipline
        if not self.user_init.wait_for_init():
            return
        self.audio.speak(text=GAME_EXPLANATION, filename="explain_game.mp3")
        self.video.process_video(output_path=os.path.join(ENVIRONMENT_DIR, "processed_video.mp4"))
        self.qna.start_loop(
            detected_object_path=self.video.detected_object_path,
            detected_object_name = self.video.detected_object_name
        )
```

# Funcionalidades

## Funcionalidad 1

**Conteo: "¿Qué objetos hay en la imagen?" → listar objetos detectados.**

Lo implementamos en *VideoManager.process_video()*, iteramos sobre todos los frames muestreados del video y usamos el detector YOLO *self.detector.detect(frame)*. Luego almacenamos los nombres de todos los objetos únicos detectados en el set llamado *self.detected_object_names*. El número de objetos detectados, *detected_object_names_len*, lo utilizamos posteriormente para dar una pista específica en *QnAEngine.process_clue()*.

```log
2025-12-12 10:38:42,134 - app_logger - DEBUG - QnAEngine: process_clue(): clue key: objects
2025-12-12 10:38:42,134 - app_logger - DEBUG - QnAEngine: process_clue(): clue_text: Aquí tienes una pista, atento. A continuación te dire los 14 posibles objetos: televisión, florero, cuenco, cama, maleta, microondas, libro, cepillo de dientes, sofá, comedor, baño, reloj, silla, banco


## Funcionalidad 2

**Selección: "Márcame el ``[objeto]``" → dibujar bounding box o resaltado.**

Lo implementamos en dos etapas:

- **Detección y Anotación en Video:** Nuestro método *VideoManager.__annotate_frame()* se encarga de dibujar los bounding boxes y las etiquetas de clase/confianza sobre cada frame en el que se detecta un objeto. Luego, *VideoManager.process_video()* genera un video de salida con todos los frames anotados, simulando nuestra interfaz de AR.

- **Guardado del Objeto Secreto:** En nuestro método *VideoManager.__save_select_detection()* seleccionamos un objeto de un frame al azar (elegido con pesos basados en la confianza de la detección) y guardamos una imagen recortada de ese objeto (el "objeto secreto") en *detected_object_path*. Esto simula el "resaltado" al aislar el objeto de interés.

<div style="text-align:center;">
    <img src="resources/images/detected_frame.png" style="width:500px;">
</div>


## Funcionalidad 3

**Preguntas: "¿Hay una ``[objeto]``?"→ respuesta sí/no con evidencia visual.**

Lo implementamos de forma avanzada, es decir, usamos el modelo **CLIP** para hacer **matching** entre la imagen del objeto detectado y las preguntas del usuario. En nuestro bucle de Q&A, la respuesta del sistema se basa en la similitud entre el texto de la pregunta (o las pistas) y la imagen, lo que equivale a responder si "existe" o "se relaciona" nuestra respuesta con la escena visual.

También lo implementamos al analizar la respuesta del usuario, es decir, si el usuario ha acertado el objeto misterioso. En este caso estamos comparando si la respuesta corresponde con un objeto que estaba en un frame cocnreto.

```log
2025-12-12 10:35:16,594 - app_logger - DEBUG - QnAEngine: process_question(): question:  Es una silla.
2025-12-12 10:35:16,594 - app_logger - DEBUG - QnAEngine: while question_idx < len(questions_files): success silla

## Funcionalidad 4

**Matching CLIP texto-imagen → buscar lo más parecido a una palabra/frase.**

Lo tenemos implementado utilizando el **CLIPProcessor** de forma intensiva tanto para generar respuestas, cada dos preguntas, como para dar pistas, cada seis preguntas. En ambos casos, el sistema busca la mayor correlación entre la imagen del objeto y una lista de textos para tomar una decisión.


<div style="text-align:center;">
    <img src="resources/images/detected_object.png" style="width:200px;">
</div>


```log
2025-12-12 10:38:36,484 - app_logger - DEBUG - QnAEngine: process_question(): question 4
2025-12-12 10:38:36,703 - app_logger - DEBUG - QnAEngine: process_question(): question:  ¿Cirbe para descansar?
2025-12-12 10:38:36,703 - app_logger - DEBUG - QnAEngine: process_question(): question 5
2025-12-12 10:38:36,886 - app_logger - DEBUG - QnAEngine: process_question(): question:  Es oscuro.
2025-12-12 10:38:36,886 - app_logger - DEBUG - QnAEngine: process_answer(): answer 3
2025-12-12 10:38:37,539 - app_logger - DEBUG - QnAEngine: process_answer(): answer: cirbe descansar

## Funcionalidad 5

**Vídeo (opcional con puntuación extra).**

Mediante *VideoManager.process_video()* simulamos el comportamiento en tiempo real de unas gafas de realidad aumentada. Es decir por cada frame estamos analizando todos los objetos contenidos en él. Luego, realizamos una selección, de forma aleatoria, de un frame que contenga objetos. A partir de aquí, nuestro problema se resuelve a un probleam de imágenes.

Es decir, estamos utilizando el vídeo para analizar el entorno y quedarnos con un frame concreto.

<div style="text-align:center;">
    <video width="600" controls>
        <source src="resources/videos/environment/processed_video.mp4" type="video/mp4">
        Tu navegador no soporta video.
    </video>
</div>
